# Import libraries

In [5]:
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import preprocessing
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as patches
from scipy.special import expit
import itertools

# Load csv file

In [6]:
dataset = pd.read_csv('/Users/mel/Documents/MRes/Machine Learning/Assignment/AMLS_20-21_SN12345678/datasets/celeba/labels.csv', sep='\\t', engine='python')

dataset.head()

,img_name,gender,smiling
0,0.jpg,-1,1
1,1.jpg,-1,1
2,2.jpg,1,-1
3,3.jpg,-1,-1
4,4.jpg,-1,-1


# Reading in the images

In [7]:
dataset_image = []
for i in tqdm(range(dataset.shape[0])):
    img = image.load_img('/Users/mel/Documents/MRes/Machine Learning/Assignment/AMLS_20-21_SN12345678/datasets/celeba/img/'+dataset['img_name'][i])
    img = img.convert('L')
    img = image.img_to_array(img)
    # normalize the data (max pixel size is 255 - so all data is between 0-1)
    img = img/255
    # confirm the normalization
    #print('Min: %.3f, Max: %.3f' % (img.min(), img.max()))
#convert from 4D to 1D
    img = img.flatten() 

    dataset_image.append(img)


100%|██████████| 5000/5000 [00:16<00:00, 305.15it/s]


In [8]:
X = np.array(dataset_image)
X.shape

(5000, 38804)

In [9]:
Y = dataset['gender']

In [10]:
# Evaluate using a train and a test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=100)
model = LogisticRegression(solver = 'lbfgs', max_iter=10000)
model.fit(X_train, Y_train)
result = model.score(X_test, Y_test)
print("Accuracy: %.2f%%" % (result*100.0))

KeyboardInterrupt: 

In [9]:
result = model.score(X_train, Y_train)
print("Accuracy: %.2f%%" % (result*100.0))

Accuracy: 100.00%


# Evaluate using cross-validation techniques

In [10]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [55]:
kfold = KFold(n_splits=5, random_state=0, shuffle=True)
model_kfold = LogisticRegression(max_iter = 20000)
results_kfold = cross_val_score(model_kfold, X, Y, cv=kfold)
print("Accuracy: %.2f%%" % (results_kfold.mean()*100.0))

Accuracy: 87.62%


In [60]:
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits=3, random_state=100, shuffle=True)
model_skfold = LogisticRegression(max_iter = 20000)
results_skfold = cross_val_score(model_skfold, X, Y, cv=skfold)
print("Accuracy: %.2f%%" % (results_skfold.mean()*100.0))

Accuracy: 87.74%


In [11]:
kfold = KFold(n_splits=10, random_state=0, shuffle=True)
model_kfold = LogisticRegression(max_iter = 20000)
results_kfold = cross_val_score(model_kfold, X, Y, cv=kfold)
print("Accuracy: %.2f%%" % (results_kfold.mean()*100.0))

Accuracy: 87.74%


In [ ]:
# example of grid searching key hyperparametres for logistic regression
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))